In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("https://raw.githubusercontent.com/casbdai/notebooks2023/main/Module3/99_CAS_BDAI_CUP/train.csv")
train.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,brand,ID
0,Tiguan,2020,29999,Automatic,2728,Diesel,145,47.9,2.0,VW,40452
1,Up,2018,8750,Manual,12500,Petrol,145,64.2,1.0,VW,33997
2,2 Series,2019,22646,Manual,123,Petrol,145,52.3,1.5,BMW,24566
3,1 Series,2019,21950,Manual,5013,Petrol,145,44.1,1.5,BMW,30438
4,X3,2013,13100,Automatic,64455,Diesel,160,50.4,2.0,BMW,22552


In [3]:
train.set_index("ID", inplace=True)
train["age"] = 2023 - train["year"]
train.drop("year", axis=1, inplace=True)

train['mileage_log'] = np.log1p(train['mileage'])
train['mileage_age_interaction'] = train['mileage'] * train['age']

train.head()

,model,price,transmission,mileage,fuelType,tax,mpg,engineSize,brand,age,mileage_log,mileage_age_interaction
ID,,,,,,,,,,,,
40452,Tiguan,29999,Automatic,2728,Diesel,145,47.9,2.0,VW,3,7.911691,8184
33997,Up,8750,Manual,12500,Petrol,145,64.2,1.0,VW,5,9.433564,62500
24566,2 Series,22646,Manual,123,Petrol,145,52.3,1.5,BMW,4,4.820282,492
30438,1 Series,21950,Manual,5013,Petrol,145,44.1,1.5,BMW,4,8.519989,20052
22552,X3,13100,Automatic,64455,Diesel,160,50.4,2.0,BMW,10,11.073738,644550


In [4]:
train["brand"].value_counts()

VW          9480
Mercedes    8169
Audi        6676
BMW         6532
Name: brand, dtype: int64

In [5]:
train_brands = train["brand"].unique()

for brand in train_brands:
    print(brand)
    print(train[train["brand"] == brand]["fuelType"].value_counts())

VW
Petrol    5552
Diesel    3757
Hybrid     111
Other       60
Name: fuelType, dtype: int64
BMW
Diesel    4336
Petrol    2001
Hybrid     173
Other       22
Name: fuelType, dtype: int64
Audi
Diesel    3445
Petrol    3211
Hybrid      20
Name: fuelType, dtype: int64
Mercedes
Diesel    5700
Petrol    2365
Hybrid      99
Other        5
Name: fuelType, dtype: int64


In [6]:
test = pd.read_csv("https://raw.githubusercontent.com/casbdai/notebooks2023/main/Module3/99_CAS_BDAI_CUP/test.csv")
test.head()

,model,year,transmission,mileage,fuelType,tax,mpg,engineSize,brand,ID
0,Golf,2015,Manual,40000,Petrol,205,39.8,2.0,VW,1
1,GLA Class,2017,Automatic,30112,Petrol,145,43.5,2.0,Mercedes,4
2,1 Series,2015,Semi-Auto,19168,Petrol,145,49.6,1.6,BMW,10
3,B Class,2019,Semi-Auto,14700,Petrol,145,45.6,1.3,Mercedes,13
4,A3,2010,Manual,64800,Petrol,200,40.9,1.6,Audi,16


In [7]:
test.set_index("ID", inplace=True)
test["age"] = 2023 - test["year"]
test.drop("year", axis=1, inplace=True)

test['mileage_log'] = np.log1p(test['mileage'])
test['mileage_age_interaction'] = test['mileage'] * test['age']

test.head()

,model,transmission,mileage,fuelType,tax,mpg,engineSize,brand,age,mileage_log,mileage_age_interaction
ID,,,,,,,,,,,
1,Golf,Manual,40000,Petrol,205,39.8,2.0,VW,8,10.596660,320000
4,GLA Class,Automatic,30112,Petrol,145,43.5,2.0,Mercedes,6,10.312712,180672
10,1 Series,Semi-Auto,19168,Petrol,145,49.6,1.6,BMW,8,9.861050,153344
13,B Class,Semi-Auto,14700,Petrol,145,45.6,1.3,Mercedes,4,9.595671,58800
16,A3,Manual,64800,Petrol,200,40.9,1.6,Audi,13,11.079076,842400


In [8]:
test_brands = train["brand"].unique()

for brand in test_brands:
    print(brand)
    print(test[test["brand"] == brand]["fuelType"].value_counts())

VW
Petrol    2908
Diesel    2031
Hybrid      34
Other       27
Name: fuelType, dtype: int64
BMW
Diesel    2217
Petrol     975
Hybrid      73
Other        6
Name: fuelType, dtype: int64
Audi
Diesel    1825
Petrol    1627
Hybrid       8
Name: fuelType, dtype: int64
Mercedes
Diesel    2884
Petrol    1215
Hybrid      65
Other        1
Name: fuelType, dtype: int64


In [9]:
train["fuelType"].value_counts()

Diesel    17238
Petrol    13129
Hybrid      403
Other        87
Name: fuelType, dtype: int64

Check if categorial variables are the same, otherwise we would get in trouble with dummies

In [10]:
set(train["model"].unique()).difference(set(test["model"].unique()))

set()

In [11]:
set(train["transmission"].unique()).difference(set(test["transmission"].unique()))


set()

In [12]:
set(train["fuelType"].unique()).difference(set(test["fuelType"].unique()))


set()

In [13]:
set(train["brand"].unique()).difference(set(test["brand"].unique()))

set()

In [14]:
brands = set(train["brand"].unique()).intersection(set(test["brand"].unique()))

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, make_scorer 

In [16]:
X_train = train.drop(["price"], axis=1)
y_train = train["price"]

X_test = test.copy()

In [17]:
param_grid = {
    "model__max_depth": [10, None],
    "model__n_estimators": [100, 150],
    "model__subsample": [0.7, 1],
    "model__learning_rate": [0.1],
    "model__gamma": [0, 0.1],
    "model__colsample_bytree": [0.75, 1],
}

In [18]:
import xgboost
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder


param_grid = {
    "model__n_estimators": [100], # [100, 150],
}
categorical_features = ['transmission', 'fuelType']
numerical_features = ['mileage', 'tax', 'mpg', 'engineSize', 'age', 'mileage_log', 'mileage_age_interaction']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

model = xgboost.XGBRegressor(random_state=1, tree_method='hist')

pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

model_prediction= pd.DataFrame([], columns=["ID", "price"]) 
model_grid_search_results = pd.DataFrame([], columns=["best_score", "best_params"])
for model in X_train["model"].unique():
    print(model)
    model_train = X_train[X_train["model"] == model].drop(["model", "brand"], axis=1)
    model_test = X_test[X_test["model"] == model].drop(["model", "brand"], axis=1)
    
    search = GridSearchCV(
        pipeline,
        param_grid,
        cv=5, 
        scoring='neg_root_mean_squared_error',
        verbose=1
    )

    search.fit(model_train,y_train[X_train["model"] == model])
    predicted_df = pd.DataFrame({"ID": X_test[X_test["model"] == model].index, "price": search.predict(model_test)})
    model_prediction = pd.concat([model_prediction, predicted_df])
    
    result_df = pd.DataFrame({"best_score": [search.best_score_], "best_params": [search.best_params_]}, index=[model])
    model_grid_search_results = pd.concat([model_grid_search_results, result_df])


 Tiguan
Fitting 5 folds for each of 1 candidates, totalling 5 fits
 Up
Fitting 5 folds for each of 1 candidates, totalling 5 fits
 2 Series
Fitting 5 folds for each of 1 candidates, totalling 5 fits
 1 Series
Fitting 5 folds for each of 1 candidates, totalling 5 fits
 X3
Fitting 5 folds for each of 1 candidates, totalling 5 fits
 Arteon
Fitting 5 folds for each of 1 candidates, totalling 5 fits
 Q5
Fitting 5 folds for each of 1 candidates, totalling 5 fits
 Passat
Fitting 5 folds for each of 1 candidates, totalling 5 fits
 Golf
Fitting 5 folds for each of 1 candidates, totalling 5 fits
 A6
Fitting 5 folds for each of 1 candidates, totalling 5 fits
 E Class
Fitting 5 folds for each of 1 candidates, totalling 5 fits
 Polo
Fitting 5 folds for each of 1 candidates, totalling 5 fits
 3 Series
Fitting 5 folds for each of 1 candidates, totalling 5 fits
 Q3
Fitting 5 folds for each of 1 candidates, totalling 5 fits
 4 Series
Fitting 5 folds for each of 1 candidates, totalling 5 fits
 5 Series


In [19]:
model_prediction.head()

,ID,price
0,87,26980.339844
1,280,25507.867188
2,346,17615.902344
3,398,8198.387695
4,422,19881.533203


In [20]:
model_grid_search_results.head()

,best_score,best_params
Tiguan,-2398.723504,{'model__n_estimators': 100}
Up,-734.261398,{'model__n_estimators': 100}
2 Series,-4251.564115,{'model__n_estimators': 100}
1 Series,-1425.287595,{'model__n_estimators': 100}
X3,-2909.249926,{'model__n_estimators': 100}


In [21]:
model_grid_search_results["best_score"].mean()

-2561.972779206235

In [22]:
model_grid_search_results["support"] = X_train["model"].value_counts() / len(X_train)
(model_grid_search_results["best_score"] * model_grid_search_results["support"]).sum()

-2107.3115403307575

In [23]:
param_grid = {
    "model__n_estimators": [100],
}
categorical_features = ['transmission', 'fuelType', "model"]
numerical_features = ['mileage', 'tax', 'mpg', 'engineSize', 'age', 'mileage_log', 'mileage_age_interaction']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

model = xgboost.XGBRegressor(random_state=1, tree_method='hist')

pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

brand_prediction= pd.DataFrame([], columns=["ID", "price"])
brand_grid_search_results = pd.DataFrame([], columns=["best_score", "best_params"]) 

for brand in X_train["brand"].unique():
    print(brand)
    brand_train = X_train[X_train["brand"] == brand].drop(["brand"], axis=1)
    brand_test = X_test[X_test["brand"] == brand].drop(["brand"], axis=1)
    
    search = GridSearchCV(
        pipeline,
        param_grid,
        cv=5, 
        scoring='neg_root_mean_squared_error',
        verbose=1
    )

    search.fit(brand_train,y_train[X_train["brand"] == brand])
    predicted_df = pd.DataFrame({"ID": X_test[X_test["brand"] == brand].index, "price": search.predict(brand_test), "score": [search.best_score_]*len(brand_test), "best_params": [search.best_params_]*len(brand_test)})
    brand_prediction = pd.concat([brand_prediction, predicted_df])

    result_df = pd.DataFrame({"best_score": [search.best_score_], "best_params": [search.best_params_]}, index=[brand])
    brand_grid_search_results = pd.concat([brand_grid_search_results, result_df])

VW
Fitting 5 folds for each of 1 candidates, totalling 5 fits
BMW
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Audi
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Mercedes
Fitting 5 folds for each of 1 candidates, totalling 5 fits


In [24]:
brand_grid_search_results["support"] = X_train["brand"].value_counts() / len(X_train)
(brand_grid_search_results["best_score"] * brand_grid_search_results["support"]).sum()

-2144.893296816683

In [25]:
param_grid = {
    "model__n_estimators": [100, 150],
}
categorical_features = ['transmission', 'fuelType', "model", "brand"]
numerical_features = ['mileage', 'tax', 'mpg', 'engineSize', 'age', 'mileage_log', 'mileage_age_interaction']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

model = xgboost.XGBRegressor(random_state=1, tree_method='hist')

pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

overall_prediction= pd.DataFrame([], columns=["ID", "price", "score", "best_params"]) 

search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5, 
    scoring='neg_root_mean_squared_error',
    verbose=1
)

search.fit(X_train,y_train)
predicted_df = pd.DataFrame({"ID": X_test.index, "price": search.predict(X_test), "score": [search.best_score_]*len(X_test), "best_params": [search.best_params_]*len(X_test)})
overall_prediction = pd.concat([overall_prediction, predicted_df])

overall_grid_search_results = pd.DataFrame({"best_score": [search.best_score_], "best_params": [search.best_params_]})

Fitting 5 folds for each of 2 candidates, totalling 10 fits


In [26]:
overall_grid_search_results

,best_score,best_params
0,-2175.069887,{'model__n_estimators': 150}


In [27]:
model_prediction.set_index("ID", inplace=True)
brand_prediction.set_index("ID", inplace=True)
overall_prediction.set_index("ID", inplace=True)

In [28]:
final_prediction = test[["model", "brand"]].copy()
final_prediction["overall_price"] = overall_prediction["price"]
final_prediction["model_price"] = model_prediction["price"]
final_prediction["brand_price"] = brand_prediction["price"]

In [29]:
final_prediction = pd.merge(final_prediction, model_grid_search_results[["best_score"]], how="left", left_on=["model"], right_index=True).rename(columns={"best_score": "model_best_score"})
final_prediction = pd.merge(final_prediction, brand_grid_search_results[["best_score"]], how="left", left_on=["brand"], right_index=True).rename(columns={"best_score": "brand_best_score"})
final_prediction["overall_best_score"] = overall_grid_search_results["best_score"][0]
final_prediction

,model,brand,overall_price,model_price,brand_price,model_best_score,brand_best_score,overall_best_score
ID,,,,,,,,
1,Golf,VW,16830.769531,16572.367188,16968.117188,-1343.180184,-1495.526917,-2175.069887
4,GLA Class,Mercedes,23520.474609,20040.195312,22939.107422,-1410.803877,-2554.314896,-2175.069887
10,1 Series,BMW,13120.172852,13853.331055,14718.910156,-1425.287595,-2558.505832,-2175.069887
13,B Class,Mercedes,20621.080078,20545.683594,21144.763672,-1333.384493,-2554.314896,-2175.069887
16,A3,Audi,7029.932129,6733.372559,6012.466309,-1763.122243,-2161.326964,-2175.069887
...,...,...,...,...,...,...,...,...
46738,4 Series,BMW,15797.873047,15737.682617,15931.091797,-2486.475085,-2558.505832,-2175.069887
46744,Touareg,VW,11175.986328,14391.787109,9483.806641,-3373.514575,-1495.526917,-2175.069887
46746,A Class,Mercedes,12938.733398,11677.121094,12452.236328,-1870.871801,-2554.314896,-2175.069887


In [31]:
final_prediction["model_weight"] = final_prediction[["model_best_score", "brand_best_score", "overall_best_score"]].max(axis=1) / final_prediction["model_best_score"] 
final_prediction["brand_weight"] = final_prediction[["model_best_score", "brand_best_score", "overall_best_score"]].max(axis=1) / final_prediction["brand_best_score"] 
final_prediction["overall_weight"] = final_prediction[["model_best_score", "brand_best_score", "overall_best_score"]].max(axis=1) / final_prediction["overall_best_score"]

In [32]:
final_prediction["price"] = (final_prediction["model_weight"] * final_prediction["model_price"] + final_prediction["brand_weight"] * final_prediction["brand_price"] + final_prediction["overall_weight"] * final_prediction["overall_price"]) / final_prediction[["model_weight", "brand_weight", "overall_weight"]].sum(axis=1)

In [33]:
final_prediction

,model,brand,overall_price,model_price,brand_price,model_best_score,brand_best_score,overall_best_score,model_weight,brand_weight,overall_weight,price
ID,,,,,,,,,,,,
1,Golf,VW,16830.769531,16572.367188,16968.117188,-1343.180184,-1495.526917,-2175.069887,1.000000,0.898132,0.617534,16777.087502
4,GLA Class,Mercedes,23520.474609,20040.195312,22939.107422,-1410.803877,-2554.314896,-2175.069887,1.000000,0.552322,0.648625,21793.317146
10,1 Series,BMW,13120.172852,13853.331055,14718.910156,-1425.287595,-2558.505832,-2175.069887,1.000000,0.557078,0.655284,13854.130487
13,B Class,Mercedes,20621.080078,20545.683594,21144.763672,-1333.384493,-2554.314896,-2175.069887,1.000000,0.522013,0.613031,20713.805572
16,A3,Audi,7029.932129,6733.372559,6012.466309,-1763.122243,-2161.326964,-2175.069887,1.000000,0.815759,0.810605,6600.986783
...,...,...,...,...,...,...,...,...,...,...,...,...
46738,4 Series,BMW,15797.873047,15737.682617,15931.091797,-2486.475085,-2558.505832,-2175.069887,0.874760,0.850133,1.000000,15820.112977
46744,Touareg,VW,11175.986328,14391.787109,9483.806641,-3373.514575,-1495.526917,-2175.069887,0.443314,1.000000,0.687576,11050.888623
46746,A Class,Mercedes,12938.733398,11677.121094,12452.236328,-1870.871801,-2554.314896,-2175.069887,1.000000,0.732436,0.860143,12314.667515


In [42]:
def save_submission_for_kaggle(model_name, test_predictions, test):
  import pandas as pd
  submission_data = pd.DataFrame({"ID": test["ID"], "Actual": test_predictions})
  submission_data.to_csv(model_name, index=False)

In [43]:
save_submission_for_kaggle("pgo_submission.csv", final_prediction["price"], final_prediction.reset_index())

In [48]:
final_prediction.reset_index()[["ID", "price"]].rename({"price": "Actual"}, axis=1).to_csv("pgo_submission.csv", index=False)

In [46]:
final_prediction["price"]

ID
1        16777.087502
4        21793.317146
10       13854.130487
13       20713.805572
16        6600.986783
             ...     
46738    15820.112977
46744    11050.888623
46746    12314.667515
46748    20146.362658
46750     9238.117136
Name: price, Length: 15896, dtype: float64

In [40]:
final_prediction["price"]

ID
1        16777.087502
4        21793.317146
10       13854.130487
13       20713.805572
16        6600.986783
             ...     
46738    15820.112977
46744    11050.888623
46746    12314.667515
46748    20146.362658
46750     9238.117136
Name: price, Length: 15896, dtype: float64

## Comparision with ground truth done in class

In [216]:
submission_data = final_prediction[["price"]].copy()

In [79]:
ground_truth = pd.read_csv("ground_truth.csv")
ground_truth.set_index("ID", inplace=True)

In [217]:
submission_data["actual"] = ground_truth["price"]
mean_squared_error(submission_data["actual"], submission_data["price"], squared=False)

1929.5365934683473

In [218]:
submission_data["model_price"] = final_prediction["model_price"]
submission_data["brand_price"] = final_prediction["overall_price"]
submission_data["overall_price"] = final_prediction["brand_price"]
print("Model: ", mean_squared_error(submission_data["actual"], submission_data["model_price"], squared=False))
print("Brand: ", mean_squared_error(submission_data["actual"], submission_data["brand_price"], squared=False))
print("Overall: ", mean_squared_error(submission_data["actual"], submission_data["overall_price"], squared=False))

Model:  2205.071411790387
Brand:  2019.607614421647
Overall:  2006.150972352351
